Here to learn the memory Management in LLM

In [48]:
import ollama
import os
import traceback
import ollama

# Langchain Core/Community imports
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# Ensure AIMessage is imported
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnablePassthrough

# Ollama/Chroma imports
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_chroma import Chroma
from chromadb import PersistentClient

ollmaclient = ollama.Client(host="http://localhost:11434/")
print("Ollama client initialized.")
print(ollmaclient.list().models[0].model)
llm_model_name = "mistral"
llm = ChatOllama(model=llm_model_name, temperature=0.1)
# print(llm.invoke("Hello, world!"))

Ollama client initialized.
qllama/bge-reranker-large:latest


In [49]:
# !pip install wikipedia

In [50]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [51]:
# wikipedia.run("What is the capital of France?")

In [52]:
!huggingface-cli whoami


Akashkansal


In [53]:
tools = [wikipedia]
llm_with_tools = llm.bind_tools(tools)
print("LLM with tools initialized.")

LLM with tools initialized.


Create Prompt Tempelate with history

In [54]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [55]:
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
# from langchain.agents.output_parsers import AgentOutputParser
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableMap
from langchain_community.chat_models import ChatOllama

# Simple output parser
class SimpleAgentOutputParser(ToolsAgentOutputParser):
    def parse(self, text):
        return text.strip()

# Agent pipeline
agent = (
    RunnableMap({
        "input": lambda x: x["input"],
        "chat_history": lambda x: x.get("chat_history", []),
        "agent_scratchpad": lambda x: format_to_tool_messages(x.get("intermediate_steps", [])),
    })
    | prompt
    | llm
    | SimpleAgentOutputParser()
)

In [56]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [ ]:
from langchain.memory import ChatMessageHistory

chat_history = ChatMessageHistory()
# query = "What is the capital of India?"
query = "Who won the 2018 Fifa Wordcup?"

response  = agent_executor({"input": query, "chat_history": chat_history.messages})
print("Response:", response) 
chat_history.add_user_message(query)
if isinstance(response, dict):
    response_content = response.get('output', '')  # Adjust key as needed
else:
    response_content = response

chat_history.add_ai_message(response_content)
print("Updated chat history:", chat_history.messages)

In [ ]:

query = "when france won the first world cup?"
response = agent_executor({"input": query, "chat_history": chat_history.messages})
print("Response:", response['output'])